# Full Pipeline (Clean)
This notebook runs recognition + generation end-to-end.


# Setup
Find the repo root dynamically and install dependencies if needed.


In [ ]:
import os
from pathlib import Path

def find_repo_root(start: Path | None = None) -> Path:
    start = (start or Path.cwd()).resolve()
    for path in [start, *start.parents]:
        if (path / 'src').is_dir():
            return path
    raise FileNotFoundError('Could not locate repo root containing src/')

repo_root = find_repo_root()
os.chdir(repo_root)
repo_root


## Section A: Setup


In [ ]:
# Optional: install dependencies
# !pip install -r requirements.txt

# Set your coordinate data root
COORD_ROOT = '/path/to/coordinate_files'  # update this


## Section B: Build manifests + sanity check


In [ ]:
MANIFEST_DIR = repo_root / 'outputs' / 'manifests'
!python scripts/build_coord_manifest.py --coord_root $COORD_ROOT --out_dir $MANIFEST_DIR
!python scripts/viz_sequences.py --npy $(ls -1 $COORD_ROOT/*/*.npy | head -n 1) --out figures/example.png


## Section C: Train recognition + evaluate test


In [ ]:
!python -m src.recognition.train_from_coords --coord_root $MANIFEST_DIR --out_dir outputs/recognition


## Section D: Train generator + sample images


In [ ]:
!python -m src.generation.train_lstm --rec_out $MANIFEST_DIR --out_dir outputs/generation
!python -m src.generation.sample_condlstm --ckpt outputs/generation/best_condlstm.pt --out_dir figures


## Section E: Evaluate generated images with recognizer


In [ ]:
!python -m src.generation.eval_with_recognizer --rec_ckpt outputs/recognition/best_recognizer.pt --gen_root figures/gen_samples --out_dir outputs/eval
